In [1]:
import requests
from datetime import datetime
import pandas as pd

In [2]:
df=pd.read_csv('df7.csv')

In [3]:
import json
def get_codeforces_rating_history(username):
    url = f"https://codeforces.com/api/user.rating?handle={username}"
    response = requests.get(url)
    
    if response.status_code == 200:
        try:
            data = response.json()
            if data["status"] == "OK":
                return data["result"]
            else:
                print(f"API Error: {data.get('comment', 'No specific error message provided')}")
        except json.JSONDecodeError:
            print("Invalid JSON response from the API")
    else:
        print(f"Failed to fetch rating history for {username}. Status code: {response.status_code}")

    return None

def days_to_cross_margin(username, target_lower, target_upper):
    rating_history = get_codeforces_rating_history(username)
    
    if rating_history is None:
        return None

    current_rating = None
    start_date = None
    end_date = None

    for entry in rating_history:
        rating = entry["newRating"]
        contest_time = entry["ratingUpdateTimeSeconds"]
        contest_date = datetime.fromtimestamp(contest_time)

        if rating <= target_lower:
            start_date = contest_date
        elif rating >= target_upper:
            end_date = contest_date
            break

    if start_date and end_date:
        days_taken = (end_date - start_date).days
        return days_taken
    else:
        print(f"{username} did not cross the rating margin within the specified range.")
        return None

def add_result_to_dataframe(df):
    df['Result'] = ""
    target_lower = 1200
    target_upper = 1400

    for index, row in df.iterrows():
        username = row['Handle']
        days_taken = days_to_cross_margin(username, target_lower, target_upper)
        
        if days_taken is not None:
            if 20 <= days_taken <= 40:
                df.at[index, 'Result'] = "Yes"
            else:
                df.at[index, 'Result'] = "No"
        else:
            df.at[index, 'Result'] = "N/A"

    return df

if __name__ == "__main__":
    
    df = add_result_to_dataframe(df)

    print(df)

Eleganty did not cross the rating margin within the specified range.
Failed to fetch rating history for NimaAryan. Status code: 504
Hazzu did not cross the rating margin within the specified range.
ZiXeL did not cross the rating margin within the specified range.
daniilzimin4 did not cross the rating margin within the specified range.
Failed to fetch rating history for temp_tamora. Status code: 503
Failed to fetch rating history for Black_Be4rD. Status code: 503
Failed to fetch rating history for DoverHi. Status code: 503
bashNewbie did not cross the rating margin within the specified range.
jskhirtladze did not cross the rating margin within the specified range.
Failed to fetch rating history for kushalgupta87. Status code: 503
Failed to fetch rating history for _LONE__WOLF_. Status code: 503
ahmad_alghadban did not cross the rating margin within the specified range.
Failed to fetch rating history for areason. Status code: 503
Failed to fetch rating history for lightblue_pikachu. Stat

In [4]:
df.head()

,Handle,Max Rating,Result
0,vuhuydiet,1784,No
1,wang_by,1830,Yes
2,1660,1783,No
3,2021404292,1783,Yes
4,Aman113114_Bansal,1857,No


In [5]:
df.to_csv('outdf7.csv', index=False)